In [1]:
import pandas as pd
import googlemaps
import os
from dotenv import load_dotenv, find_dotenv


In [2]:
# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()
# load up the entries as environment variables
load_dotenv(dotenv_path)
# get the path of data
path_interim_data = os.environ.get("PATH_INTERIM_DATA_CLEANED")
path_processed_data = os.environ.get("PATH_PROCESSED_DATA")
# get google maps key
gmaps_key = os.environ.get("GOOGLE_MAPS_KEY")

# set filename
filename = 'crimes.csv'

# set country 
country = 'BR'

# set city name
city = 'São Carlos'


In [23]:
# load data
path_data = path_interim_data + filename
data = pd.read_csv(path_data,index_col='index')
noref = data['LATITUDE'] == 0
data_noref = data[noref]
data_noref['Num_logradouro'] = data_noref['Num_logradouro'].astype(int).astype(str)
data_noref['CEP'] = data_noref['CEP'].astype(str) 
data_noref.fillna('',inplace=True)
data_noref.replace('nan', '', inplace=True)
data_noref.head()

C:\Users\Tiago\Anaconda3\envs\virtualenv\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Tiago\Anaconda3\envs\virtualenv\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Tiago\Anaconda3\envs\virtualenv\lib\site-packages\pandas\core\frame.py:4263: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/p

,Ano,Num_BO,Data_ocorrencia_BO,Hora_ocorrencia_BO,Data_hora_registro_BO,Data_hora_impressao_BO,Período,Flagrante,Status,Conduta,Bairro,Tipo_local,CEP,LOGRADOURO,Num_logradouro,LATITUDE,LONGITUDE
index,,,,,,,,,,,,,,,,,
11,2017,644,2017-09-21,18:00:00,2017-09-23 13:27:17.000000,2017-09-23 13:54:17.000000,,N,Consumado,TRANSEUNTE,,RESTAURANTE E AFINS,,AVENIDA GETULIO VARGAS,1,0.0,0.0
16,2019,596,2019-02-05,10:15:00,2019-02-06 01:18:47.000000,2019-02-06 03:11:05.000000,,N,Consumado,VEICULO,RURAL,RODOVIA/ESTRADA,,RODOVIA SP 310,265,0.0,0.0
23,2017,9,2017-01-18,,2017-01-18 16:08:03.000000,2017-01-20 17:09:43.000000,DE MADRUGADA,N,Consumado,VEICULO,,RODOVIA/ESTRADA,,RODOVIA WASHINGTON LUIZ KM 221,221,0.0,0.0
24,2015,1276,2015-12-20,,2015-12-21 12:15:28.000000,2015-12-21 12:37:30.000000,EM HORA INCERTA,N,Consumado,VEICULO,,VIA PÚBLICA,,RUA CLAUDENICE BALDAN MAIELO,227,0.0,0.0
34,2014,970,2014-08-11,18:00:00,2014-08-12 11:38:57.000000,2014-08-12 14:36:44.000000,,N,Consumado,VEICULO,DISTRITO INDUSTRIAL,VIA PÚBLICA,14820000,AVENIDA HERMINIO CRISTOVÃO,395,0.0,0.0


In [24]:
gmaps = googlemaps.Client(key=gmaps_key, queries_per_second=40)

components = {
    'country': country,
    'administrative_area': city
}

In [25]:
no_results = []
more_than_one = []
partial_match = []

for index, row in data_noref.iterrows():
    try:
            
            address = row.LOGRADOURO + ',' + row.Num_logradouro + ',' + row.Bairro + ',' + row.CEP + ',' + city
            print(address)
            result = gmaps.geocode(language='pt-BR', address= address, components=components)
            data_noref.loc[index, 'LATITUDE'] = result[0]['geometry']['location']['lat']
            data_noref.loc[index, 'LONGITUDE'] = result[0]['geometry']['location']['lng']
    except Exception as e:
        print(e)

AVENIDA GETULIO VARGAS,1,,,São Carlos
RODOVIA SP 310,265,RURAL,,São Carlos
RODOVIA WASHINGTON LUIZ  KM 221,221,,,São Carlos
RUA CLAUDENICE BALDAN MAIELO,227,,,São Carlos
AVENIDA HERMINIO CRISTOVÃO,395,DISTRITO INDUSTRIAL,14820000,São Carlos
AVENIDA JOAQUIM PINTO PEREIRA DE ALMEIDA,585,CENTRO,14820000,São Carlos
RUA PERIMETRAL PROJETADA,81,IV DISTRITO INDUSTRIAL,14820000,São Carlos
RUA ALEXANDRE PEDRAZZANI,190,,,São Carlos
RUA PADRE TEIXEIRA - CLUBE FLOR DE MAIO,1,VL MAX,13561050,São Carlos
RUA SECOMBINA DE PAULA PASSADOR,393,CIDADE ARACY,13573090,São Carlos
RUA ANTONIO SELARIM,80,CENTRO,-,São Carlos
RUA NUNCIO CARDINALI,4,RURAL,,São Carlos
RUA GERMANO FEHR JUNIOR,435,CASTELO BRANCO,,São Carlos
R CAP LUIZ BRANDAO,291,VL STA MARIA II,,São Carlos
RUA RUI BARBOSA,102,VILA MONTEIRO,,São Carlos
RUA MAJOR MANOEL ANTONIO DE MATTOS,195,VILA MONTEIRO,,São Carlos
RUA JOSÉ LEMES MARQUES,170,VILA SÃO JOSÉ,,São Carlos
RUA PADRE TEIXEIRA,2431,VILA NERY,,São Carlos
RUA FAGUNDES VARELA,305,VILA MARCELI

RUA GASTÃO DE SÁ,1169,VILA PRADO,13574002,São Carlos
RUA LUIZ OLAI,189,ARACI II,13560000,São Carlos
RUA JOSE BENETTI,263,V PRADO,,São Carlos
RUA ELIDIA BENETTI,1584,JARDIM BEATRIZ,,São Carlos
RUA JOAQUIM SOLFA,60,,,São Carlos
AVENIDA PAULO VI,570,JARDIM CRUZEIRO DO SUL,13560000,São Carlos
AVENIDA REGIT ARABE,95,CIDADE ARACY,13560000,São Carlos
RUA FRANCISCO PARROTA,253,MONTE CARLO,,São Carlos
RUA ISAAC FALGEM,1414,,,São Carlos
RUA JAIME BRUNO,70,ANTENOR GARCIA,,São Carlos
AVENIDA REGITE HARAB,101,CIDADE ARACY,,São Carlos
RUA GIOVANE VASSOLO,415,MONTE CARLO,,São Carlos
AVENIDA ANTONIO MAROTTI,790,CEAT,13573480,São Carlos
REGIT HARABE,701,ARACY,,São Carlos
RUA DOIS,525,JARDIM ZAVAGLIA,13560000,São Carlos
RUA VANDELEI QUELE DE LIMA,20,PARQUE NOVO MUNDO,,São Carlos
RUA ORESTES MASTRO FRANCISCO,240,CIDADE ARACY I,,São Carlos
RUA JOSÉ PEREIRA LOPES,1871,JD BOTAFOGO,,São Carlos
RUA ANTIGA SEIS,580,VILA PRADO,,São Carlos
RUA QUARENTA E CINCO,387,ARACY II,,São Carlos
RUA CRISTOVÃO REIS,46,BELA 

AVENIDA COM. ALFREDO MAFFEI,1,CENTRO,,São Carlos
RUA EPISCOPAL,570,CENTRO,,São Carlos
RUA NOVE DE JULHO,1261,CENTRO,,São Carlos
RUA PADRE TEIXEIRA,994,,,São Carlos
AVENIDA SAO CARLOS,1,CENTRO,,São Carlos
RUA RIO ARAGUAIA,607,JOQUEI CLUBE,,São Carlos
RUA DONA ALEXANDRINA,1,CENTRO,,São Carlos
RUA SALGADO FILHO X AV.SÃO CARLOS,1,CIDADE JARDIM,,São Carlos
RUA SAO JOAQUIM,1,CENTRO,,São Carlos
RUA JOAO DE GUZZI,3675,SANTA FELICIA,,São Carlos
RUA ITALO SAVELLI,169,ARNON DE MELO,,São Carlos
AVENIDA SAO CARLOS,1,CENTRO,,São Carlos
RUA SANTA CRUZ,1,CENTRO,,São Carlos
AVENIDA ROMUALDO VILLANI,98,JD. IPANEMA,,São Carlos
AVENIDA COMENDADOR ALFREDO MAFFEI,1,JD. GILBERTONI,,São Carlos
AVENIDA TANCREDO ALMEIDA NEVES,477,BOTOFOGO,,São Carlos
RUA SANTA CRUZ,1,CENTRO,,São Carlos
RUA PROCOPIO TOLEDO MALTA,1,SANTA FELICIA,,São Carlos
RUA SERAFIM VIEIRA DE ALMEIDA,184,,,São Carlos
RUA RAY WESLEY HERRIKY,1,JOQUEY CLUBE,,São Carlos
RUA COMENDADOR ALFREDO MAFFEI,2230,,,São Carlos
AVENIDA JOÃO DE GUZZI,25,SANTA

AV MORUMBI,866,JD CRUZEIRO DO SUL,,São Carlos
RUA MAL. DEODORO,1324,CENTRO,13560000,São Carlos
RUA MARECHAL DEODORO,3183,VILA NEREY,,São Carlos
RUA BAHIA, DEF,369,JD CRUZEIRO DO SUL,,São Carlos
RUA SOFIA BAGNATO,192,DISTRITO INDUSTRIAL,,São Carlos
AV JOSÉ PEREIRA LOPES,676,VILA PRADO,,São Carlos
RUA JOÃO ANTONIO BONI,1189,J. SÃO JOÃO BATISTA,,São Carlos
RUA PROF. EUVIDIO GOUVEIA,10,BOA VISTA,,São Carlos
RUA IWAGIRO TOYAMA,51,SANTA PAULA,,São Carlos
RUA ANGELO LUCATO,18,PARQUE DELTA,,São Carlos
RUA EDUARDO GOBATO,183,DISTRITO INDUSTRIAL,,São Carlos
RUA CLOVIS PACHECO,60,SANTA FELICIA,,São Carlos
RUA CAP. ALBERTO MENDES JUNIOR,346,TIJUCO PRETO,,São Carlos
AV. TETRACAMPEONATO,186,ARACY II,,São Carlos
RUA IWAGIRO TOYAMA,585,JD PAULISTANO,,São Carlos
RUA IWAGIRO TOYAMA,585,JD PAULISTANO,,São Carlos
RUA MAJOR JOSÉ INÁCIO,3014,VILA NERY,,São Carlos
RUA JOSE BENETTI,707,VILA PRADO,,São Carlos
AV. GETULIO VARGAS,1250,JD. LUTFALA,,São Carlos
RUA PROFESSORA ELIDIA BENETI,1254,REDENÇÃO,,São Carlos

RUA THEODOLINA MODENA COCA,218,VILA NERY,,São Carlos
ESTRADA MUNICIPAL JOSE VENANCIO,1,JD EMBARÉ,,São Carlos
RUA LUIS AUGUSTO DE OLIVEIRA,10,VILA MARINA,,São Carlos
RUA 135,145,ARACI II,,São Carlos
RUA CLEMETE TALARICO,915,,,São Carlos
RUA JOAQUIM GARCIA OLIVEIRA,57,ARACY I,,São Carlos
AVENIDA PEDRO NEO,159,NOVO MUNDO,,São Carlos
RUA R JOAQUIM DA CRUZ PENALVA,740,NV ESTANCIA,,São Carlos
AVENIDA BRUNO RUGGIERO FILHO,1157,SANTA FELICIA,,São Carlos
ESTRADA DO BROA,1,RURAL,,São Carlos
RUA VINTE E QUATRO DE MAIO,332,CENTRO,,São Carlos
ESTRADA DO BROA,1,RURAL,,São Carlos
ESTRADA DO BROA,1,RURAL,,São Carlos
AVENIDA VICENTE PELICANO,200,CASTELO BRANCO,,São Carlos
AVENIDA BRUNO RUGIERO FILHO,1157,,,São Carlos
RUA DR PADUA SALLES,215,VL PRADO,,São Carlos
RUA JOÃO GENOVÊS,83,PROLONGAMENTO DO MEDEIROS,,São Carlos
RUA MACOLINA  LIMA  MACHADO,260,PROLONGAMENTO DO MEDEIROS,,São Carlos
RUA MACOLINA  LIMA  MACHADO,260,PROLONGAMENTO DO MEDEIROS,,São Carlos
RUA NICOLA HERCULES,731,JD ICARAI - IBATE/SP,,S

RUA DR ANTONIO PEREIRA DE NOVAES,707,VL BELA VISTA,13574033,São Carlos
PASSEIO  FLAMBOYANTS,200,CENTRO,,São Carlos
AVENIDA CEL JOSÉ AUGUSTO OLIVEIRA SALES,874,VL RANCHO VELHO,13560160,São Carlos
RUA ANTONIO LEOPOLDINO GALVÃO,59,VL BELA VISTA,13574033,São Carlos
RUA LAIRE MANFIEO,265,VL BELA VISTA,13574033,São Carlos
RODOVIA DEPUTADO VICENTE BOTTA,130,JD SOCIAL PRESIDENTE COLLOR,,São Carlos
RUA LÍDIA MIGLIATO,155,VL BELA VISTA,13574033,São Carlos
AVENIDA MIGUEL PETRONI,3400,CENTRO,,São Carlos
RUA PASSEIO DOS FLAMBOYANTS,200,CENTRO,,São Carlos
ESTRADA MUNICIPAL GUILHERME SCATENA,1,VL RANCHO VELHO,13560160,São Carlos
ALAMEDA DOS JASMIM,141,CENTRO,,São Carlos
RUA CLEMENTE TALARICO,540,CENTRO,,São Carlos
RUA 7,1,VL BELA VISTA,13574033,São Carlos
RUA PASSEIO DOS FLAMBOIANS,200,PQ FABER II,13562420,São Carlos
RUA SETE DE SETEMBRO,2400,CENTRO,,São Carlos
AVENIDA SÃO CARLOS,783,VL RANCHO VELHO,13560160,São Carlos
RUA PASSEIO DOS FLAMBOYANTS,200,JD BANDEIRANTES,13562190,São Carlos
PASSEIO  DOS Y

In [28]:
#save file
#data_noref.to_csv(path_processed_data+filename, index=False)
data.update(data_noref)
data.to_csv(path_processed_data+filename, index=False)